# Install packages and download dataset

In [ ]:
# https://huggingface.co/google-bert/bert-base-multilingual-cased

In [ ]:
!pip install --upgrade pip
!pip install sentencepiece
!pip install datasets
!pip install transformers

In [ ]:
!pip install transformers[torch]

In [ ]:
!pip install optuna

In [ ]:
!pip install accelerate -U

# Prepare dataset

In [ ]:
# Download the combined training data
# Please ensure the combined_training_esen.json file is in the data directory
# You can generate this file using the data processing notebook

Downloading...
From: https://drive.google.com/uc?id=10tmR1VyVPySlwIKbkgRkfaqofBJwTTqx
To: /content/combined_training_esen.json
100% 1.29M/1.29M [00:00<00:00, 118MB/s]


In [ ]:
import pandas as pd

# Load JSON data directly into a DataFrame
df = pd.read_json('./data/combined_training_esen.json')
len(df)

# Display the first few rows of the DataFrame to verify
print(df.head())


                                                text  label
0  No me puedo creer que me haya engañado por otr...      0
1  Cuando tu colega dice "el número de cuerpos de...      1
2  Sí, las pollas son importantes, me temo ******...      0
3  2016 TODA LA SOCIEDAD #MGTOW ¡Por favor, sal c...      1
4  El logo oficial de la zona de amigos IG: @meme...      0


# Fine-tuning seperately

## clean train and test dataset for training

In [ ]:
# len(X_english_test)

In [ ]:
X = df['text'].values
y = df['label'].values.astype(int)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
len(X)

3077

In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
import numpy as np
from datasets import load_dataset, load_metric
import transformers
from transformers import AutoTokenizer
import optuna

In [ ]:
num_labels = len(set(y))

In [ ]:
num_labels

2

In [ ]:
MODEL = "dccuchile/bert-base-spanish-wwm-cased"
MAX_TRAINING_EXAMPLES = -1

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL, use_fast=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/480k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

In [ ]:
train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True)
val_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True)

In [ ]:
import torch
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):

        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)


In [ ]:
train_dataset = MyDataset(train_encodings, y_train)
val_dataset = MyDataset(val_encodings, y_test)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    acc = accuracy_score(labels, predictions)

    f1_pos = f1_score(labels, predictions, average='binary', pos_label=1)

    precision_pos = precision_score(labels, predictions, pos_label=1)
    recall_pos = recall_score(labels, predictions, pos_label=1)

    precision_macro = precision_score(labels, predictions, average='macro')
    recall_macro = recall_score(labels, predictions, average='macro')
    f1_macro = f1_score(labels, predictions, average='macro')

    precision_micro = precision_score(labels, predictions, average='micro')
    recall_micro = recall_score(labels, predictions, average='micro')
    f1_micro = f1_score(labels, predictions, average='micro')

    conf_matrix = confusion_matrix(labels, predictions)

    return {
        "accuracy": acc,
        "f1_score_positive": f1_pos,
        "precision_positive": precision_pos,
        "recall_positive": recall_pos,
        "precision_macro": precision_macro,
        "recall_macro": recall_macro,
        "f1_macro": f1_macro,
        "precision_micro": precision_micro,
        "recall_micro": recall_micro,
        "f1_micro": f1_micro,
        "confusion_matrix": conf_matrix.tolist()
    }



def objective(trial):

    def model_init():
        model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=num_labels)
        model.classifier.dropout = torch.nn.Dropout(trial.suggest_float('dropout_rate', 0, 0.5))  # Adjust dropout
        return model

    learning_rate = trial.suggest_float('learning_rate', 1e-5, 5e-5, log=True)
    num_train_epochs = trial.suggest_int('num_train_epochs', 3, 5)
    per_device_train_batch_size = trial.suggest_categorical('per_device_train_batch_size', [8, 16])
    warmup_steps = trial.suggest_int('warmup_steps', 0, 500)
    weight_decay = trial.suggest_float('weight_decay', 0.0, 0.3)

    args = TrainingArguments(
        output_dir='./results',
        learning_rate=learning_rate,
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=per_device_train_batch_size,
        per_device_eval_batch_size=per_device_train_batch_size,
        warmup_steps=warmup_steps,
        weight_decay=weight_decay,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model='accuracy',
        greater_is_better=True,
        logging_dir='./logs',
        logging_steps=10,

    )

    trainer = Trainer(
        model_init=model_init,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
    )

    # Train model
    trainer.train()
    # Evaluate model
    eval_results = trainer.evaluate()
    print(f"Accuracy: {eval_results['eval_accuracy']}")
    print(f"F1 Score (Positive): {eval_results['eval_f1_score_positive']}")
    print(f"Macro F1 Score: {eval_results['eval_f1_macro']}")
    print(f"Micro F1 Score: {eval_results['eval_f1_micro']}")

    return eval_results['eval_accuracy']


In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=15)

[I 2024-07-01 19:45:24,866] A new study created in memory with name: no-name-635b73fe-963e-482e-af84-e2e63c174066
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1 Score Positive,Precision Positive,Recall Positive,Precision Macro,Recall Macro,F1 Macro,Precision Micro,Recall Micro,F1 Micro,Confusion Matrix
1,0.575800,0.565113,0.707792,0.775000,0.715935,0.844687,0.702230,0.675355,0.679167,0.707792,0.707792,0.707792,"[[126, 123], [57, 310]]"
2,0.497700,0.545262,0.725649,0.773154,0.761905,0.784741,0.714986,0.711648,0.713066,0.725649,0.725649,0.725649,"[[159, 90], [79, 288]]"
3,0.412600,0.584180,0.730519,0.781579,0.755725,0.809264,0.720912,0.711861,0.714942,0.730519,0.730519,0.730519,"[[153, 96], [70, 297]]"
4,0.280300,0.673696,0.741883,0.795367,0.753659,0.841962,0.736053,0.718170,0.722958,0.741883,0.741883,0.741883,"[[148, 101], [58, 309]]"
5,0.191200,0.697203,0.728896,0.778220,0.759067,0.798365,0.718664,0.712436,0.714789,0.728896,0.728896,0.728896,"[[156, 93], [74, 293]]"


Trainer is attempting to log a value of "[[126, 123], [57, 310]]" of type <class 'list'> for key "eval/confusion_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[[159, 90], [79, 288]]" of type <class 'list'> for key "eval/confusion_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[[153, 96], [70, 297]]" of type <class 'list'> for key "eval/confusion_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[[148, 101], [58, 309]]" of type <class 'list'> for key "eval/confusion_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[[156, 93], [74, 293]]" of type <class 'l

...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1 Score Positive,Precision Positive,Recall Positive,Precision Macro,Recall Macro,F1 Macro,Precision Micro,Recall Micro,F1 Micro,Confusion Matrix
1,0.577700,0.582515,0.701299,0.752022,0.744000,0.760218,0.689427,0.687338,0.688256,0.701299,0.701299,0.701299,"[[153, 96], [88, 279]]"
2,0.550000,0.538668,0.732143,0.782609,0.757653,0.809264,0.722577,0.713869,0.716886,0.732143,0.732143,0.732143,"[[154, 95], [70, 297]]"
3,0.342400,0.700169,0.727273,0.757925,0.804281,0.716621,0.722210,0.729797,0.722829,0.727273,0.727273,0.727273,"[[185, 64], [104, 263]]"
4,0.102900,1.147524,0.720779,0.761773,0.774648,0.749319,0.711079,0.714017,0.712259,0.720779,0.720779,0.720779,"[[169, 80], [92, 275]]"
5,0.053100,1.268428,0.727273,0.769231,0.775623,0.762943,0.717223,0.718821,0.717949,0.727273,0.727273,0.727273,"[[168, 81], [87, 280]]"


Trainer is attempting to log a value of "[[153, 96], [88, 279]]" of type <class 'list'> for key "eval/confusion_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[[154, 95], [70, 297]]" of type <class 'list'> for key "eval/confusion_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[[185, 64], [104, 263]]" of type <class 'list'> for key "eval/confusion_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[[169, 80], [92, 275]]" of type <class 'list'> for key "eval/confusion_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[[168, 81], [87, 280]]" of type <class 'li

Trainer is attempting to log a value of "[[154, 95], [70, 297]]" of type <class 'list'> for key "eval/confusion_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
[I 2024-07-01 23:56:56,486] Trial 14 finished with value: 0.7321428571428571 and parameters: {'learning_rate': 3.260143415695753e-05, 'num_train_epochs': 5, 'per_device_train_batch_size': 16, 'warmup_steps': 189, 'weight_decay': 0.18943883109644696, 'dropout_rate': 0.3158787651339925}. Best is trial 10 with value: 0.7564935064935064.


Accuracy: 0.7321428571428571
F1 Score (Positive): 0.7826086956521738
Macro F1 Score: 0.716885743174924
Micro F1 Score: 0.7321428571428571


In [ ]:
study.best_trial

FrozenTrial(number=10, state=TrialState.COMPLETE, values=[0.7564935064935064], datetime_start=datetime.datetime(2024, 7, 1, 22, 31, 5, 151704), datetime_complete=datetime.datetime(2024, 7, 1, 22, 47, 9, 674825), params={'learning_rate': 4.54647473506188e-05, 'num_train_epochs': 4, 'per_device_train_batch_size': 16, 'warmup_steps': 254, 'weight_decay': 0.1580808091745869, 'dropout_rate': 0.10274824157881546}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=5e-05, log=True, low=1e-05, step=None), 'num_train_epochs': IntDistribution(high=5, log=False, low=3, step=1), 'per_device_train_batch_size': CategoricalDistribution(choices=(8, 16)), 'warmup_steps': IntDistribution(high=500, log=False, low=0, step=1), 'weight_decay': FloatDistribution(high=0.3, log=False, low=0.0, step=None), 'dropout_rate': FloatDistribution(high=0.5, log=False, low=0.0, step=None)}, trial_id=10, value=None)

In [ ]:
study.best_params

{'learning_rate': 4.54647473506188e-05,
 'num_train_epochs': 4,
 'per_device_train_batch_size': 16,
 'warmup_steps': 254,
 'weight_decay': 0.1580808091745869,
 'dropout_rate': 0.10274824157881546}

In [ ]:
args = TrainingArguments(
        output_dir='./results',
        learning_rate=study.best_params["learning_rate"],
        num_train_epochs=study.best_params["num_train_epochs"],
        per_device_train_batch_size=study.best_params["per_device_train_batch_size"],
        per_device_eval_batch_size=study.best_params["per_device_train_batch_size"],
        warmup_steps=study.best_params["warmup_steps"],
        weight_decay=study.best_params["weight_decay"],
        evaluation_strategy="epoch",
        save_strategy="epoch",
        metric_for_best_model='accuracy',
        logging_dir='./logs',
        logging_steps=10,

)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
def model_init(trail=None):
        model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=num_labels)
        model.classifier.dropout = torch.nn.Dropout(study.best_params["dropout_rate"])
        return model

trainer = Trainer(
    model_init=model_init,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    #callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1 Score Positive,Precision Positive,Recall Positive,Precision Macro,Recall Macro,F1 Macro,Precision Micro,Recall Micro,F1 Micro,Confusion Matrix
1,0.578600,0.581148,0.696429,0.746955,0.741935,0.752044,0.684492,0.683251,0.683823,0.696429,0.696429,0.696429,"[[153, 96], [91, 276]]"
2,0.580000,0.536648,0.756494,0.799465,0.784777,0.814714,0.747708,0.742698,0.744774,0.756494,0.756494,0.756494,"[[167, 82], [68, 299]]"
3,0.280200,0.640352,0.733766,0.780161,0.767810,0.792916,0.723567,0.719751,0.721356,0.733766,0.733766,0.733766,"[[161, 88], [76, 291]]"
4,0.066100,1.113075,0.727273,0.777188,0.757106,0.798365,0.716981,0.710428,0.712862,0.727273,0.727273,0.727273,"[[155, 94], [74, 293]]"


Trainer is attempting to log a value of "[[153, 96], [91, 276]]" of type <class 'list'> for key "eval/confusion_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[[167, 82], [68, 299]]" of type <class 'list'> for key "eval/confusion_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[[161, 88], [76, 291]]" of type <class 'list'> for key "eval/confusion_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[[155, 94], [74, 293]]" of type <class 'list'> for key "eval/confusion_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


TrainOutput(global_step=616, training_loss=0.38907787281197387, metrics={'train_runtime': 970.5213, 'train_samples_per_second': 10.143, 'train_steps_per_second': 0.635, 'total_flos': 2590065228963840.0, 'train_loss': 0.38907787281197387, 'epoch': 4.0})

In [ ]:
trainer.evaluate()['eval_accuracy']

Trainer is attempting to log a value of "[[155, 94], [74, 293]]" of type <class 'list'> for key "eval/confusion_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


0.7272727272727273

In [ ]:
trainer.evaluate()

Trainer is attempting to log a value of "[[155, 94], [74, 293]]" of type <class 'list'> for key "eval/confusion_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 1.113074541091919,
 'eval_accuracy': 0.7272727272727273,
 'eval_f1_score_positive': 0.7771883289124669,
 'eval_precision_positive': 0.7571059431524548,
 'eval_recall_positive': 0.7983651226158038,
 'eval_precision_macro': 0.7169809191744807,
 'eval_recall_macro': 0.7104275412275807,
 'eval_f1_macro': 0.7128619468830117,
 'eval_precision_micro': 0.7272727272727273,
 'eval_recall_micro': 0.7272727272727273,
 'eval_f1_micro': 0.7272727272727273,
 'eval_confusion_matrix': [[155, 94], [74, 293]],
 'eval_runtime': 12.6387,
 'eval_samples_per_second': 48.739,
 'eval_steps_per_second': 3.086,
 'epoch': 4.0}

In [ ]:
# Download test data
# Please ensure the test_data.json file is in the data directory

Downloading...
From: https://drive.google.com/uc?id=1DAD8DanXohNMlLcpZoTNqeUTABHGM6dR
To: /content/test_data.json
100% 146k/146k [00:00<00:00, 134MB/s]


In [ ]:
import pandas as pd
import json

file_path = './data/test_data.json'
with open(file_path, 'r') as file:
    data = [json.loads(line) for line in file]

df_test = pd.DataFrame(data)

print(len(df_test))
print(df_test.head())


343
                                       combined_text  gold
0  Cuando entras a las redes sociales y te topas ...     0
1  Cuando tus canciones de rap ya llegan a 100 re...     1
2  NI VÍCTIMAS NI PASIVAS, FEMINISTAS COMBATIVAS ...     1
3  imgflip.com YO CUANDO ME EXIGEN SÉ LA PROFESIO...     1
4  intensa.mx Mira, te llamo para decirte que no ...     0


In [ ]:
X_test_final = df_test['combined_text'].values

In [ ]:
import pandas as pd
import json

file_path = './data/test_data.json'
with open(file_path, 'r') as file:
    data = [json.loads(line) for line in file]

texts = [entry['combined_text'] for entry in data]
labels = [entry['gold'] for entry in data]

df_test = pd.DataFrame({
    'combined_text': texts,
    'gold': labels,
    'predictions': [0] * len(texts)
})

print(len(df_test))
print(df_test.head())


343
                                       combined_text  gold  predictions
0  Cuando entras a las redes sociales y te topas ...     0            0
1  Cuando tus canciones de rap ya llegan a 100 re...     1            0
2  NI VÍCTIMAS NI PASIVAS, FEMINISTAS COMBATIVAS ...     1            0
3  imgflip.com YO CUANDO ME EXIGEN SÉ LA PROFESIO...     1            0
4  intensa.mx Mira, te llamo para decirte que no ...     0            0


In [ ]:
import json
from datasets import Dataset

file_path = './data/test_data.json'
with open(file_path, 'r') as file:
    data = [json.loads(line) for line in file]

texts = [entry['combined_text'] for entry in data]

dummy_labels = [0] * len(texts)
test_dict = dict({'text': texts, 'predictions': dummy_labels})
test_encodings = tokenizer(X_test_final.tolist(), truncation=True, padding=True)
test_dataset = MyDataset(test_encodings, test_dict['predictions'])

In [ ]:
test_predictions = trainer.predict(test_dataset)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
test_preds_raw, test_labels , _ = trainer.predict(test_dataset)
test_preds = np.argmax(test_preds_raw, axis=-1)
print(test_preds)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[0 0 1 0 1 1 1 0 1 0 0 1 0 1 1 1 1 0 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 0
 1 1 1 0 0 1 1 1 1 0 0 0 1 0 1 1 0 1 0 0 1 1 1 1 1 1 0 1 1 0 1 1 1 0 1 0 0
 1 0 0 1 1 1 1 1 0 0 1 1 0 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 0 1
 1 0 1 0 1 1 1 1 0 1 1 0 1 1 0 1 1 1 1 0 1 0 1 1 1 1 0 1 1 0 1 1 0 1 1 0 1
 0 0 1 1 0 1 1 0 0 0 1 1 1 1 0 1 1 1 1 1 0 1 0 1 0 1 1 1 0 0 1 1 0 0 1 1 1
 0 1 1 1 0 0 1 1 1 0 0 1 1 1 0 1 0 1 0 0 1 1 1 1 0 1 1 1 1 1 0 1 0 1 1 1 0
 1 1 1 1 0 1 1 0 0 1 0 1 0 1 0 1 1 0 1 1 0 0 0 1 1 0 0 1 0 1 1 1 0 0 1 1 1
 1 0 1 1 0 0 0 1 0 1 1 0 0 1 1 1 1 0 1 0 0 1 1 1 1 1 1 0 0 1 1 1 0 0 0 1 1
 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 0 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 0 0 1
 0 0 1 1 0 0 1 0 1 0]


In [ ]:
import numpy as np
import pandas as pd


df_predictions = pd.DataFrame({
    'text': texts,
    'predicted_label': test_preds,
    'gold_label': labels
})


df_predictions.to_csv('./results/google-bert-based-uncased-predictions-ablation.csv', index=False)

print("预测结果已经保存到 'predictions.csv' 文件.")


预测结果已经保存到 'predictions.csv' 文件.


In [ ]:
with open('./data/spanish_memes_test.json', 'r') as file:
    data_ids = json.load(file)

ids = [entry['id_EXIST'] for entry in data_ids.values()]
values = ["NO" if pred == 0 else "YES" if pred == 1 else None for pred in test_preds]

df_predictions = pd.DataFrame({
    'id': ids,
    'value': values,
    'test_case': 'EXIST2024'
})

df_predictions.to_json('./results/bert-based-uncased-es_test_submission.json', orient='records', lines=True, indent=2)


In [ ]:
len(test_preds)

343

In [ ]:
len(df_predictions)

343